## Cleaning data extracted from Sapo

In [321]:
import pandas as pd
import re

file_path = 'data/raw/raw_sapo_2024-07-25.parquet'
df = pd.read_parquet(file_path)
df.sample(5)

,type,location,link,price_euro,info_agg,offer_type,site,date_extracted,offer_type_search,property_type_search,location_search,sub_location_search
21100,Apartamento T1,"Centro Histórico (Cascais), Cascais e Estoril",https://gespub.casa.sapo.pt/v3/webinterface/cl...,1.600 €,Recuperado,alugar,sapo,2024-07-25T19:44:18.015810,alugar,apartamentos,distrito.lisboa,
7087,Apartamento T2,"Avenida 24 de Julho (Santos-o-Velho), Estrela,...",/comprar-apartamento-t2-lisboa-estrela-avenida...,649.000 €,Renovado · 115m²,comprar,sapo,2024-07-25T19:42:24.458415,comprar,apartamentos,distrito.lisboa,
19470,Moradia T2,"Alcoentre, Azambuja, Distrito de Lisboa",/comprar-moradia-t2-azambuja-alcoentre-574d88d...,80.000 €,Usado · 53m²,comprar,sapo,2024-07-25T19:43:50.672804,comprar,moradias,distrito.lisboa,
12819,Apartamento T3,"Damaia, Águas Livres, Amadora, Distrito de Lisboa",/comprar-apartamento-t3-amadora-aguas-livres-d...,445.000 €,Novo · 102m²,comprar,sapo,2024-07-25T19:42:24.458415,comprar,apartamentos,distrito.lisboa,
13115,Apartamento,"Lumiar, Lisboa, Distrito de Lisboa",/comprar-apartamento-lisboa-lumiar-5c9015b0-32...,425.000 €,Usado · 80m²,comprar,sapo,2024-07-25T19:42:24.458415,comprar,apartamentos,distrito.lisboa,


In [322]:
df.duplicated().sum()

np.int64(14)

In [323]:
df.isna().sum()

type                    0
location                0
link                    0
price_euro              0
info_agg                0
offer_type              0
site                    0
date_extracted          0
offer_type_search       0
property_type_search    0
location_search         0
sub_location_search     0
dtype: int64

In [324]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23507 entries, 0 to 23506
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   type                  23507 non-null  object
 1   location              23507 non-null  object
 2   link                  23507 non-null  object
 3   price_euro            23507 non-null  object
 4   info_agg              23507 non-null  object
 5   offer_type            23507 non-null  object
 6   site                  23507 non-null  object
 7   date_extracted        23507 non-null  object
 8   offer_type_search     23507 non-null  object
 9   property_type_search  23507 non-null  object
 10  location_search       23507 non-null  object
 11  sub_location_search   23507 non-null  object
dtypes: object(12)
memory usage: 2.2+ MB


## Splitting information

### `location` 
- Each location stays in its own column.

In [325]:
df.location.sample(5)

13094    São Bento (Santos-o-Velho), Estrela, Lisboa, D...
4272     Santa Iria de Azoia, São João da Talha e Bobad...
7414                   Estrela, Lisboa, Distrito de Lisboa
16506                Cascais e Estoril, Distrito de Lisboa
2560                            Loures, Distrito de Lisboa
Name: location, dtype: object

In [326]:
# Split location info
df_location_info_splitted = (
    df['location']
    .str.split(',', expand=True)
)
df_location_info_splitted.columns = [
    f'location_zone_{n}' for n in df_location_info_splitted.columns
]
df = df.join(df_location_info_splitted)

df_location_info_splitted.sample(5)

,location_zone_0,location_zone_1,location_zone_2,location_zone_3,location_zone_4,location_zone_5
2609,Estrela (Lapa),Lisboa,Distrito de Lisboa,None,None,None
5660,Olivais,Lisboa,Distrito de Lisboa,None,None,None
8953,Avenida Forças Armadas (Nossa Senhora de Fátima),Avenidas Novas,Lisboa,Distrito de Lisboa,None,None
15609,Serra da Amoreira (Ramada),Ramada e Caneças,Odivelas,Distrito de Lisboa,None,None
4687,Quinta das Pevides,Mafra,Distrito de Lisboa,None,None,None


### `info_agg`
This field can contain:
- Property_status + area
- Only property_status
- Only area
- Other case

In [327]:
df.info_agg.sample(10)

10624            Usado  ·  183m²
9432              Usado  ·  92m²
9972              Novo  ·  125m²
9504     Em construção  ·  553m²
4614           Renovado  ·  70m²
20778            Usado  ·  119m²
6503              Usado  ·  48m²
1223     Em construção  ·  138m²
18561            Usado  ·  124m²
1177              Novo  ·  152m²
Name: info_agg, dtype: object

In [328]:
# define patterns
pattern_no_number = r'^[^\d.]+$' 
pattern_has_number = r'\d'
pattern_area = r'^\d+m²$'

# filter number of cases
just_area = df[df['info_agg'].str.contains(pattern_area, regex=True, na=False)].shape[0]
status_and_area = df[df['info_agg'].str.contains(pattern_has_number, regex=True, na=False)].shape[0] - just_area
just_status = df[df['info_agg'].str.contains(pattern_no_number, regex=True, na=False)].shape[0]
null_info_agg = df[df.info_agg.isna()].shape[0]

print(
f'''Number of records
Total:  {df.shape[0]}
Status and Area: {status_and_area}
Just Status: {just_status}
Just Area: {just_area}
Other Case: {df.shape[0] - status_and_area - just_status - just_area}
Null: {null_info_agg}
'''
)


Number of records
Total:  23507
Status and Area: 23124
Just Status: 297
Just Area: 80
Other Case: 6
Null: 0



The main pattern is the first one, so after the split:
- `info_0`: Rename to `property_status`.
- `info_1`: Rename to `area_m2`. 

In [329]:
# Split information aggregated
df_info_agg_splitted = (
    df['info_agg']
    .str.split('·', 
    expand=True)
)
df_info_agg_splitted.columns = [
    f'info_{n}' for n in df_info_agg_splitted.columns
]
df = df.join(df_info_agg_splitted)

# rename columns
df.rename(
    columns={
        'info_0': 'property_status',
        'info_1': 'area_m2',
    }, 
    inplace= True
)

The following cases are trated:

**1.** When it has just `property_status`: 

`area_m2` is `None` and `property_status` is correct -> No action required

**2.** Whe it has just `area_m2`:

`area_m2` is in `property_status`  -> Action required

In [330]:
# Test before transformation

pattern_has_number = r'\d'
property_status_with_numbers = df[df['property_status'].str.contains(pattern_has_number, regex=True, na=False)].shape[0]

assert property_status_with_numbers != 0

In [331]:
def correct_area_and_property_status_values(row):
    pattern_has_number = r'\d'
    if (
         re.match(pattern_has_number, row['property_status'])
    ):
        row['area_m2'] = row['property_status']
        row['property_status'] = None
    return row

df = df.apply(correct_area_and_property_status_values, axis = 1)

In [332]:
# Test after transformation

pattern_has_number = r'\d'
property_status_with_numbers = df[df['property_status'].str.contains(pattern_has_number, regex=True, na=False)].shape[0]

assert property_status_with_numbers == 0

Remove leading and trailing spaces, and set empty strings to `None`.

In [333]:
df['property_status'] = df['property_status'].str.strip()
df['property_status'] = df['property_status'].replace('', None)

Checking for numbers in `property_status`

In [334]:
df.property_status.value_counts()

property_status
Usado                          11342
Novo                            5226
Em construção                   3285
Renovado                        1721
Recuperado                      1329
Por recuperar                    344
Em projecto                      126
Para Demolir ou Reconstruir       48
Name: count, dtype: int64

Converting `area_m2` to a numeric type

In [335]:
df['area_m2'] = df['area_m2'].str.replace(r'\D', '', regex=True)
df['area_m2'] = pd.to_numeric(df['area_m2'], errors= 'coerce')

In [336]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23507 entries, 0 to 23506
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  23507 non-null  object 
 1   location              23507 non-null  object 
 2   link                  23507 non-null  object 
 3   price_euro            23507 non-null  object 
 4   info_agg              23507 non-null  object 
 5   offer_type            23507 non-null  object 
 6   site                  23507 non-null  object 
 7   date_extracted        23507 non-null  object 
 8   offer_type_search     23507 non-null  object 
 9   property_type_search  23507 non-null  object 
 10  location_search       23507 non-null  object 
 11  sub_location_search   23507 non-null  object 
 12  location_zone_0       23507 non-null  object 
 13  location_zone_1       23507 non-null  object 
 14  location_zone_2       21128 non-null  object 
 15  location_zone_3    

### `type`

The number of bedrooms information is in this field.
- Extract the `num_bedrooms` from the `type` column.
- Rename the `type` column to `title`

In [337]:
pattern_num_bedrooms = r'(T\d\S*)'

rows_with_information = df[df.type.str.contains(pattern_num_bedrooms, regex = True, na = False)].shape[0]
rows_with_no_information = df[~df.type.str.contains(pattern_num_bedrooms, regex = True, na = True)].shape[0]
null_values = df[df.type.isna()].shape[0]

print(
f'''Number of bedrooms information
Total rows:             {df.shape[0]}
Has the information:    {rows_with_information}
Does not have information:  {rows_with_no_information}
Null:                   {null_values}
'''
)

Number of bedrooms information
Total rows:             23507
Has the information:    23045
Does not have information:  462
Null:                   0



/tmp/ipykernel_51735/2758145169.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  rows_with_information = df[df.type.str.contains(pattern_num_bedrooms, regex = True, na = False)].shape[0]
/tmp/ipykernel_51735/2758145169.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  rows_with_no_information = df[~df.type.str.contains(pattern_num_bedrooms, regex = True, na = True)].shape[0]


Extracting `num_bedrooms` from `type` column

In [338]:
pattern_num_bedrooms = r'(T\d\S*)'
df['num_bedrooms'] = df['type'].str.extract(pattern_num_bedrooms)

Checking for records and `NaN` values in `num_bedrooms`.

In [339]:
# number of records extracted
assert df[df.num_bedrooms.notna()].shape[0] == rows_with_information
assert df[df.num_bedrooms.isna()].shape[0] == rows_with_no_information

Transform records that contain a `+` into the actual sum of the bedrooms.

In [340]:
records_before = df[df.num_bedrooms.notna()].shape[0]

def sum_num_bedrooms(record):
    if (
        isinstance(record, str) 
        and '+' in record
        and len(record) == 4
    ):
        begin_str, end_str = record.split('+')
        real_sum = int(end_str) + int(begin_str[-1])
        return f'T{real_sum}'
    return record

df['num_bedrooms'] = df['num_bedrooms'].apply(sum_num_bedrooms)

Transform values with more than 9 bedrooms to `T9+`

In [341]:
values_accepted = [ f'T{n}' for n in range(10)]
values_accepted.append('T9+')

def normalize_num_bedrooms(record):
    if (
        isinstance(record, str)
        and record not in values_accepted
    ):
        return 'T9+'
    return record

df['num_bedrooms'] = df['num_bedrooms'].apply(normalize_num_bedrooms)

In [342]:
df.num_bedrooms.value_counts()

num_bedrooms
T3     7274
T2     6514
T4     3874
T1     2695
T5     1306
T6      520
T0      431
T7      173
T8      110
T9       79
T9+      69
Name: count, dtype: int64

Checking if we have the same number of records

In [320]:
assert df[df.num_bedrooms.notna()].shape[0] == records_before

Rename `type` column to `title`

In [108]:
# rename type columns
df.rename(
    columns={
        'type': 'title',
    }, 
    inplace= True
)

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23507 entries, 0 to 23506
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 23507 non-null  object 
 1   location              23507 non-null  object 
 2   link                  23507 non-null  object 
 3   price_euro            23507 non-null  object 
 4   info_agg              23507 non-null  object 
 5   offer_type            23507 non-null  object 
 6   site                  23507 non-null  object 
 7   date_extracted        23507 non-null  object 
 8   offer_type_search     23507 non-null  object 
 9   property_type_search  23507 non-null  object 
 10  location_search       23507 non-null  object 
 11  sub_location_search   23507 non-null  object 
 12  location_zone_0       23507 non-null  object 
 13  location_zone_1       23507 non-null  object 
 14  location_zone_2       21128 non-null  object 
 15  location_zone_3    

## Change data types

In [110]:
# Change the wrong column name
try:
    df.rename(columns={'prince': 'price_euro'}, inplace= True)
except:
    pass

try:
    df.rename(columns={'price': 'price_euro'}, inplace= True)
except:
    pass

# remove time in date
df['date_extracted'] = pd.to_datetime(df['date_extracted']).dt.date

# change price column
df['price_euro'] = df['price_euro'].str.replace(r'\D', '', regex=True)
df['price_euro'] = pd.to_numeric(df['price_euro'], errors= 'coerce')

# Set none in empty strings in sub_location_search
df['sub_location_search'] = (
    df['sub_location_search']
    .apply(lambda x: None if x == '' else x)
)

## Drop columns and duplicates

In [111]:
# Remove location and info_agg
df = df.drop(
    columns= ['location', 'info_agg']
)

# Drop Duplicated
df.drop_duplicates(inplace=True)

## Set ID from link
- There is an ID in the end of each link, like this: `/comprar-alugar-apartamento-t6-ou-superior-lisboa-estrela-711abb47-7f1c-11ee-aeeb-060000000051.html`. The ID will be this `711abb47-7f1c-11ee-aeeb-060000000051`
- The ID will be used to identify each advertisement.
- We must check for duplicated ID's and ensure that all of them start with `ID`.
- Some links do not contain this `link` structure, so in these cases, the `link_id` will be `NaN`.

In [112]:
df.link.sample(10)

11657    /comprar-apartamento-t2-lisboa-sao-domingos-de...
14631    /comprar-apartamento-t2-lisboa-avenidas-novas-...
15815    /comprar-moradia-t2-lisboa-marvila-748ef50b-11...
20438    /comprar-moradia-t5-cascais-e-estoril-a2d63f87...
6283     /comprar-apartamento-t4-amadora-aguas-livres-u...
16508    /comprar-moradia-t4-mafra-ericeira-20ed85ae-77...
17794    /comprar-moradia-geminada-t3-cascais-carcavelo...
2384     /comprar-apartamento-t2-cascais-sao-domingos-d...
13175    /comprar-apartamento-t1-cascais-e-estoril-d450...
11598    /comprar-apartamento-t4-cascais-e-estoril-c00a...
Name: link, dtype: object

In [113]:
df[df.link.duplicated()].shape[0]

36

There are some duplicated IDs. This happens because some properties are avaiable for both purchase and rent (*comprar/alugar*). In these cases, only the purchase option will be kept because:
1. This is the price pattern
2. This is the first search 

In [114]:
df[
    (df.duplicated(subset='link'))
    & (df.price_euro < 30000)
]

,title,link,price_euro,offer_type,site,date_extracted,offer_type_search,property_type_search,location_search,sub_location_search,location_zone_0,location_zone_1,location_zone_2,location_zone_3,location_zone_4,location_zone_5,property_status,area_m2,num_bedrooms
23261,Villa,/comprar-alugar-villa-lisboa-sao-vicente-graca...,8500.0,comprar / alugar,sapo,2024-07-25,alugar,moradias,distrito.lisboa,None,Graça,São Vicente,Lisboa,Distrito de Lisboa,None,None,Recuperado,260.0,NaN


Removing duplicate links

In [115]:
index_to_remove = df[
    (df.link.duplicated(keep='first'))
    & (df.offer_type_search == 'alugar')
].index

df.drop(index=index_to_remove, inplace=True)

Checking for duplicates

In [116]:
assert df[df.link.duplicated()].shape[0] == 0

Retrieving ID information from `link`. It extracts the 32 characters preceding *.html* in `link`.

In [117]:
pattern = r'([a-zA-Z0-9\-]{32})\.html$'
df['link_id'] = df['link'].str.extract(pattern)
# Convert to strings
df['link_id'] = df['link_id'].astype(str)

Checking the cases where there is no ID in the `link` column, so the `link_id` is `NaN`,

In [118]:
assert df[(df.link_id.apply(len) < 32)].shape[0] == df[df.link.str.contains('http')].shape[0]

Setting null values where the string is *'nan'*

In [119]:
df['link_id'] = df['link_id'].replace('nan', None)

Checking for duplicated `link_id` values

In [120]:
assert df.link_id.nunique() == df.link_id.notna().sum()

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23457 entries, 0 to 23506
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 23457 non-null  object 
 1   link                  23457 non-null  object 
 2   price_euro            23050 non-null  float64
 3   offer_type            23457 non-null  object 
 4   site                  23457 non-null  object 
 5   date_extracted        23457 non-null  object 
 6   offer_type_search     23457 non-null  object 
 7   property_type_search  23457 non-null  object 
 8   location_search       23457 non-null  object 
 9   sub_location_search   0 non-null      object 
 10  location_zone_0       23457 non-null  object 
 11  location_zone_1       23457 non-null  object 
 12  location_zone_2       21084 non-null  object 
 13  location_zone_3       5930 non-null   object 
 14  location_zone_4       681 non-null    object 
 15  location_zone_5       86

## Reindex the dataframe

In [122]:
# Remove columns
desired_columns = [
    'title', 
    'price_euro', 
    'area_m2',
    'num_bedrooms',
    'property_status',
    'location_zone_0',
    'location_zone_1',
    'location_zone_2',
    'location_zone_3', 
    'location_zone_4',
    'location_zone_5', 
    'location_zone_6', 
    'site',
    'link_id',
    'link', 
    'offer_type_search', 
    'property_type_search',
    'location_search',
    'sub_location_search',
    'date_extracted'
]

df = df.reindex(columns=desired_columns)

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23457 entries, 0 to 23506
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 23457 non-null  object 
 1   price_euro            23050 non-null  float64
 2   area_m2               23161 non-null  float64
 3   num_bedrooms          22996 non-null  object 
 4   property_status       23371 non-null  object 
 5   location_zone_0       23457 non-null  object 
 6   location_zone_1       23457 non-null  object 
 7   location_zone_2       21084 non-null  object 
 8   location_zone_3       5930 non-null   object 
 9   location_zone_4       681 non-null    object 
 10  location_zone_5       86 non-null     object 
 11  location_zone_6       0 non-null      float64
 12  site                  23457 non-null  object 
 13  link_id               22767 non-null  object 
 14  link                  23457 non-null  object 
 15  offer_type_search     23

## Checking transformation

In [124]:
(df.isna().sum() / df.shape[0]) * 100

title                     0.000000
price_euro                1.735090
area_m2                   1.261883
num_bedrooms              1.965298
property_status           0.366628
location_zone_0           0.000000
location_zone_1           0.000000
location_zone_2          10.116383
location_zone_3          74.719700
location_zone_4          97.096815
location_zone_5          99.633372
location_zone_6         100.000000
site                      0.000000
link_id                   2.941553
link                      0.000000
offer_type_search         0.000000
property_type_search      0.000000
location_search           0.000000
sub_location_search     100.000000
date_extracted            0.000000
dtype: float64

In [125]:
df.duplicated().sum()

np.int64(0)

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23457 entries, 0 to 23506
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 23457 non-null  object 
 1   price_euro            23050 non-null  float64
 2   area_m2               23161 non-null  float64
 3   num_bedrooms          22996 non-null  object 
 4   property_status       23371 non-null  object 
 5   location_zone_0       23457 non-null  object 
 6   location_zone_1       23457 non-null  object 
 7   location_zone_2       21084 non-null  object 
 8   location_zone_3       5930 non-null   object 
 9   location_zone_4       681 non-null    object 
 10  location_zone_5       86 non-null     object 
 11  location_zone_6       0 non-null      float64
 12  site                  23457 non-null  object 
 13  link_id               22767 non-null  object 
 14  link                  23457 non-null  object 
 15  offer_type_search     23

In [127]:
df.sample(5)

,title,price_euro,area_m2,num_bedrooms,property_status,location_zone_0,location_zone_1,location_zone_2,location_zone_3,location_zone_4,location_zone_5,location_zone_6,site,link_id,link,offer_type_search,property_type_search,location_search,sub_location_search,date_extracted
18721,Moradia T4,845000.0,450.0,T4,Usado,Santa Iria de Azoia,São João da Talha e Bobadela,Loures,Distrito de Lisboa,None,None,NaN,sapo,7ba5-0157-11ef-8c6f-060000000054,/comprar-moradia-t4-loures-santa-iria-de-azoia...,comprar,moradias,distrito.lisboa,None,2024-07-25
10161,Apartamento T2,200000.0,70.0,T2,Usado,Mina de Água,Amadora,Distrito de Lisboa,None,None,None,NaN,sapo,b69a-cf22-11ee-adb9-060000000054,/comprar-apartamento-t2-amadora-mina-de-agua-4...,comprar,apartamentos,distrito.lisboa,None,2024-07-25
4304,Apartamento T2,245000.0,71.0,T2,Usado,Marvila,Lisboa,Distrito de Lisboa,None,None,None,NaN,sapo,190b-49a3-11ef-8add-060000000054,/comprar-apartamento-t2-lisboa-marvila-80f0190...,comprar,apartamentos,distrito.lisboa,None,2024-07-25
20838,Moradia Isolada T5,1100000.0,383.0,T5,Novo,Pontinha e Famões,Odivelas,Distrito de Lisboa,None,None,None,NaN,sapo,5a6d-8f48-11ee-a521-060000000056,/comprar-moradia-isolada-t5-odivelas-pontinha-...,comprar,moradias,distrito.lisboa,None,2024-07-25
19302,Andar de Moradia T2,137700.0,51.0,T2,Usado,Arruda dos Vinhos,Distrito de Lisboa,None,None,None,None,NaN,sapo,a42e-f177-11ee-8c6f-060000000054,/comprar-andar-de-moradia-t2-arruda-dos-vinhos...,comprar,moradias,distrito.lisboa,None,2024-07-25
